# SENTINEL-2  EOPF Zarr TO Healpix GRID CONVERSION

This notebook converts Sentinel-2 reflectance data from UTM projection 
to H3 hexagonal grid system for spatial analysis

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import healpy as hp
from tqdm import tqdm
import h3
from pyproj import Transformer
import xdggs

1. Open data and preparation

In [2]:
# Open EOPF zarr product
zarr_path = "S2A_MSIL2A_20240511T001111_N0510_R073_T56KKB_20240511T025552.zarr"
dt = xr.open_datatree(zarr_path, engine="zarr", mask_and_scale=False, chunks={})
ds = dt.measurements.reflectance.r60m
ds

<xarray.DataTree 'r60m'>
Group: /measurements/reflectance/r60m
    Dimensions:  (y_60m: 1830, x_60m: 1830)
    Coordinates:
        x        (x_60m) float32 7kB dask.array<chunksize=(305,), meta=np.ndarray>
        y        (y_60m) float32 7kB dask.array<chunksize=(305,), meta=np.ndarray>
    Dimensions without coordinates: y_60m, x_60m
    Data variables:
        b01      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b02      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b03      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b04      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b05      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b06      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b07      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b09      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b11      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b12      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
        b8a      (y_60m, x_60m) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>

2. Coordinate transformation preparation

In [3]:
# Extract Band 02 (Blue) at 60m resolution
band = "b02"
# Generate mesh of x/y coordinates
x = ds[band]['x'].values
y = ds[band]['y'].values
xx, yy = np.meshgrid(x, y)
print(f"Coordinate grid shape: {xx.shape}")
print(f"X range: {x.min():.0f} to {x.max():.0f}")
print(f"Y range: {y.min():.0f} to {y.max():.0f}")

Coordinate grid shape: (1830, 1830)
X range: 200010 to 309750
Y range: 7590250 to 7699990


In [4]:
# Setup coordinate transformation from UTM to WGS84 lat/lon
utm_crs = dt.other_metadata['horizontal_CRS_code']
transformer = Transformer.from_crs(utm_crs, "EPSG:4326", always_xy=True)
lon, lat = transformer.transform(xx, yy)
# Convert UTM coordinates to latitude/longitude
lon, lat = transformer.transform(xx, yy)
level=16
nside=2**(level)

idx=hp.ang2pix(nside,lon,lat,lonlat=True,nest=True)
lidx,ilidx=np.unique(idx,return_inverse=True)
imh=np.bincount(ilidx.flatten())

h_lon,h_lat=hp.pix2ang(nside,lidx.flatten(),lonlat=True,nest=True)

In [5]:
from numcodecs import Zstd
import numpy as np
import xarray as xr

class proj_odysea:
    """
    HEALPix projection class for spatial data aggregation compatible with xdggs
    """

    def __init__(
        self,
        level,
        heal_idx,
        inv_idx,
        nscale=2,
        nest=False,
        chunk_size=4096,
        cell_id_name="cell_ids",
    ):
        self.level = level
        self.nside = 2**(level)
        self.nscale = nscale
        self.nest = nest
        self.chunk_size = chunk_size
        self.cell_id_name = cell_id_name

        # HEALPix cell setup with ONLY xdggs-compatible attributes
        self.cell_ids = heal_idx.flatten()
        self.var_cell_ids = xr.DataArray(
            self.cell_ids,
            dims="cells",
            attrs={
                "grid_name": "healpix",
                "indexing_scheme": "nested" if self.nest else "ring",
                "resolution": self.level,
                # Remove ALL legacy attributes that cause conflicts
            }
        )
        self.inv_idx = inv_idx.flatten()
        self.him = np.bincount(self.inv_idx)

    def eval(self, ds):
        """
        Convert dataset to HEALPix projection without time dimension
        """
        var_name = list(ds.data_vars)
        print(f"Processing {len(var_name)} variables: {var_name}")

        # Initialize 2D data array (bands, cells)
        all_data = np.zeros([len(var_name), self.cell_ids.shape[0]])

        # Process each variable
        for i in range(len(var_name)):
            ivar = var_name[i]
            print(f"Processing {ivar} ({i+1}/{len(var_name)})")

            # Flatten spatial data
            b_data = ds[ivar].values.flatten()

            # Find valid data (non-zero and non-NaN)
            idx = np.where((b_data != 0) & (~np.isnan(b_data)))

            # Aggregate to HEALPix cells
            data = np.bincount(
                self.inv_idx[idx],
                weights=b_data[idx],
                minlength=self.cell_ids.shape[0]
            )

            # Count pixels per cell
            hdata = np.bincount(
                self.inv_idx[idx],
                minlength=self.cell_ids.shape[0]
            )

            # Calculate mean (handle division by zero)
            data = data.astype(float)
            data[hdata == 0] = np.nan
            valid_mask = hdata > 0
            data[valid_mask] = data[valid_mask] / hdata[valid_mask]

            # Store in 2D array
            all_data[i] = data

        # Create DataArray with correct dimensions
        data_array = xr.DataArray(
            all_data,
            dims=("bands", "cells"),
            coords={
                "bands": var_name,
                self.cell_id_name: self.var_cell_ids
            },
            name='Sentinel2',
            attrs={
                "description": "Sentinel-2 reflectance aggregated to HEALPix cells"
            }
        )

        # Convert to Dataset
        ds_total = data_array.to_dataset()

        # Set ONLY xdggs-compatible attributes (no extra attributes)
        ds_total[self.cell_id_name].attrs = {
            "grid_name": "healpix",
            "indexing_scheme": "nested" if self.nest else "ring",
            "resolution": self.level,
        }

        # Apply chunking
        chunk_size_data = max(1, int((12 * (4**self.level)) / self.chunk_size))
        ds_total = ds_total.chunk({"cells": chunk_size_data})

        print(f"✅ HEALPix conversion complete - Level {self.level}, {len(self.cell_ids):,} cells")

        return ds_total



In [6]:
from numcodecs import Zstd

nside=2**level
nside_list=2**level
nest=True
chunk_size = 4096 #12 *(2**level)
chunk_size_data = int(( 12 * (4**level)) / chunk_size)
print("chunk_size", chunk_size)

zstd_compressor = Zstd(level=3)

# Define a common compression setting
compression_settings = {"compressor": zstd_compressor,}

chunk_size 4096


In [7]:
pr=proj_odysea(level, lidx, ilidx, nest=nest, chunk_size= chunk_size, nscale=1)

In [8]:
ds_healpix =pr.eval(ds)

Processing 11 variables: ['b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b09', 'b11', 'b12', 'b8a']
Processing b01 (1/11)
Processing b02 (2/11)
Processing b03 (3/11)
Processing b04 (4/11)
Processing b05 (5/11)
Processing b06 (6/11)
Processing b07 (7/11)
Processing b09 (8/11)
Processing b11 (9/11)
Processing b12 (10/11)
Processing b8a (11/11)
✅ HEALPix conversion complete - Level 16, 1,221,781 cells


In [9]:
ds = ds_healpix.pipe(xdggs.decode)
ds

<xarray.Dataset> Size: 117MB
Dimensions:    (bands: 11, cells: 1221781)
Coordinates:
  * bands      (bands) <U3 132B 'b01' 'b02' 'b03' 'b04' ... 'b11' 'b12' 'b8a'
  * cell_ids   (cells) int64 10MB dask.array<chunksize=(1221781,), meta=np.ndarray>
Dimensions without coordinates: cells
Data variables:
    Sentinel2  (bands, cells) float64 108MB dask.array<chunksize=(11, 1221781), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=16, indexing_scheme=nested)

In [10]:
cell_centers = ds.dggs.cell_centers()

In [11]:
derived_ds = ds_healpix.assign_coords(
    cell_centers.rename_vars({"latitude": "lat", "longitude": "lon"}).coords
)
derived_ds

<xarray.Dataset> Size: 137MB
Dimensions:    (bands: 11, cells: 1221781)
Coordinates:
  * bands      (bands) <U3 132B 'b01' 'b02' 'b03' 'b04' ... 'b11' 'b12' 'b8a'
    cell_ids   (cells) int64 10MB dask.array<chunksize=(1221781,), meta=np.ndarray>
    lat        (cells) float64 10MB -21.78 -21.78 -21.78 ... -20.78 -20.78
    lon        (cells) float64 10MB 150.9 150.9 150.9 ... 150.2 150.2 150.2
Dimensions without coordinates: cells
Data variables:
    Sentinel2  (bands, cells) float64 108MB dask.array<chunksize=(11, 1221781), meta=np.ndarray>

In [12]:
cell_boundaries = ds.dggs.cell_boundaries()
cell_boundaries

<xarray.DataArray (cells: 1221781)> Size: 10MB
array([<POLYGON ((150.9 -21.779, 150.901 -21.779, 150.9 -21.778, 150.899 -21.779, 1...>,
       <POLYGON ((150.902 -21.779, 150.903 -21.778, 150.902 -21.777, 150.901 -21.77...>,
       <POLYGON ((150.901 -21.779, 150.901 -21.778, 150.901 -21.777, 150.9 -21.778,...>,
       ...,
       <POLYGON ((150.156 -20.777, 150.156 -20.777, 150.156 -20.776, 150.155 -20.77...>,
       <POLYGON ((150.156 -20.777, 150.157 -20.776, 150.156 -20.775, 150.156 -20.77...>,
       <POLYGON ((150.155 -20.777, 150.156 -20.776, 150.155 -20.775, 150.154 -20.77...>],
      shape=(1221781,), dtype=object)
Coordinates:
    cell_ids  (cells) int64 10MB dask.array<chunksize=(1221781,), meta=np.ndarray>
Dimensions without coordinates: cells

In [13]:
ds

<xarray.Dataset> Size: 117MB
Dimensions:    (bands: 11, cells: 1221781)
Coordinates:
  * bands      (bands) <U3 132B 'b01' 'b02' 'b03' 'b04' ... 'b11' 'b12' 'b8a'
  * cell_ids   (cells) int64 10MB dask.array<chunksize=(1221781,), meta=np.ndarray>
Dimensions without coordinates: cells
Data variables:
    Sentinel2  (bands, cells) float64 108MB dask.array<chunksize=(11, 1221781), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(level=16, indexing_scheme=nested)

In [14]:
def create_arrow_table(polygons, arr, coords=None):
    from arro3.core import Array, ChunkedArray, Schema, Table

    if coords is None:
        coords = ["latitude", "longitude"]

    array = Array.from_arrow(polygons)
    name = arr.name or "data"
    arrow_arrays = {
        "geometry": array,
        "cell_ids": ChunkedArray([Array.from_numpy(arr.coords["cell_ids"])]),
        name: ChunkedArray([Array.from_numpy(arr.data)]),
    } | {
        coord: ChunkedArray([Array.from_numpy(arr.coords[coord].data)])
        for coord in coords
        if coord in arr.coords
    }

    fields = [array.field.with_name(name) for name, array in arrow_arrays.items()]
    schema = Schema(fields)

    return Table.from_arrays(list(arrow_arrays.values()), schema=schema)


def normalize(var, center=None):
    from matplotlib.colors import CenteredNorm, Normalize

    if center is None:
        vmin = var.min(skipna=True)
        vmax = var.max(skipna=True)
        normalizer = Normalize(vmin=vmin, vmax=vmax)
    else:
        halfrange = np.abs(var - center).max(skipna=True)
        normalizer = CenteredNorm(vcenter=center, halfrange=halfrange)

    return normalizer(var.data)


def exploire_layer(
    arr,
    cell_dim="cells",
    cmap="viridis",
    center=None,
    alpha=None,
):
    from lonboard import SolidPolygonLayer
    from lonboard.colormap import apply_continuous_cmap
    from matplotlib import colormaps

    if len(arr.dims) != 1 or cell_dim not in arr.dims:
        raise ValueError(
            f"exploration only works with a single dimension ('{cell_dim}')"
        )

    cell_ids = arr.dggs.coord.data
    grid_info = arr.dggs.grid_info

    polygons = grid_info.cell_boundaries(cell_ids, backend="geoarrow")

    normalized_data = normalize(arr.variable, center=center)

    colormap = colormaps[cmap]
    colors = apply_continuous_cmap(normalized_data, colormap, alpha=alpha)

    table = create_arrow_table(polygons, arr)
    layer = SolidPolygonLayer(table=table, filled=True, get_fill_color=colors)

    return layer

In [15]:
ds.Sentinel2

<xarray.DataArray 'Sentinel2' (bands: 11, cells: 1221781)> Size: 108MB
dask.array<xarray-Sentinel2, shape=(11, 1221781), dtype=float64, chunksize=(11, 1221781), chunktype=numpy.ndarray>
Coordinates:
  * bands     (bands) <U3 132B 'b01' 'b02' 'b03' 'b04' ... 'b11' 'b12' 'b8a'
  * cell_ids  (cells) int64 10MB dask.array<chunksize=(1221781,), meta=np.ndarray>
Dimensions without coordinates: cells
Indexes:
    cell_ids  HealpixIndex(level=16, indexing_scheme=nested)
Attributes:
    description:  Sentinel-2 reflectance aggregated to HEALPix cells

In [16]:
import lonboard

#use of tanh to concentrate the scale variation for the lower values
ds1=ds.Sentinel2.sel(bands='b04').compute()

lonboard.Map(
    [
        exploire_layer(
            ds1,
            alpha=0.33,
            cmap='jet'
        )
    ]
)

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…